# AI 學分班 - 作業

本作業使用 [Kaggle Titanic](https://www.kaggle.com/c/titanic/overview) 所提供的資料，根據鐵達尼號乘客資料預測生還者。
我們只使用 `train.csv` 進行生還者預測（只有 `train.csv` 才有答案），同學可以額外使用 `test.csv` 進行模型預測後提交至 [Kaggle Titanic](https://www.kaggle.com/c/titanic/overview) 進行評分。

## 評分標準

如果有參考Kaggle上的程式碼或是其他作法卻沒有附上來源，則直接視為瓢竊，得分為0分。

### 得分規則
- 程式 (60%)，請於此檔案進行程式撰寫，完成後繳交此檔案

|規則|滿足規則得分|
|-|-|
|使用決策樹及決策樹以外的模型進行實驗|+10|
|使用 5-fold cross validation 輸出平均準確度|+10|
|使用80%訓練資料訓練模型、以20%資料進行驗證，並輸出準確度|+10|
|嘗試兩個以上的資料填補策略|+10|
|在使用相同驗證集的情況下，嘗試使用不同比例資料訓練模型，並觀察結果|+10|
|嘗試使用不同的特徵作為模型輸入 (例如：使用 `Sex` 與 `Age` 以外的特徵）|+10|

- 報告 (40%)，請用 Microsoft Word 進行撰寫，完成後繳交 .docx 或 .pdf 檔案。

|規則|滿足規則得分|
|-|-|
|描述資料分析內容 (例如：使用 `.describe()` 觀察資料)|+5|
|描述選擇特定特徵輸入的理由 (例如：使用 `Sex` 與 `Age` 以外的特徵)|+5|
|描述處理缺失值的方法與理由|+5|
|描述轉換數值資料的方法與理由|+5|
|描述轉換類別資料的方法與理由|+5|
|以資料視覺化輔助資料分析並進行內容描述|+5|
|說明使用的模型並解釋模型超參數的選擇理由|+10|

In [ ]:
# 請勿更動此區塊程式碼

import time
import numpy as np
import pandas as pd

EXECUTION_START_TIME = time.time() # 計算執行時間

df = pd.read_csv('train.csv')      # 讀取資料，請勿更改路徑

## 資料前處理

In [ ]:
# 資料分析與前處理

train_x = df[['Sex', 'Age']]                   # 取出訓練資料需要分析的資料欄位
train_y = df['Survived']                       # 取出訓練資料的答案

from sklearn.impute import SimpleImputer       # 匯入填補缺失值的工具
from sklearn.preprocessing import LabelEncoder # 匯入 Label Encoder

imputer = SimpleImputer(strategy='median')     # 創造 imputer 並設定填補策略
age = train_x['Age'].to_numpy().reshape(-1, 1)
imputer.fit(age)                               # 根據資料學習需要填補的值
train_x['Age'] = imputer.transform(age)        # 填補缺失值

le = LabelEncoder()                            # 創造 Label Encoder
le.fit(train_x['Sex'])                         # 給予每個類別一個數值
train_x['Sex'] = le.transform(train_x['Sex'])  # 轉換所有類別成為數值

## 模型訓練

請在此區塊後使用**一個或多個區塊**進行模型訓練：

- **選擇模型**
    - 設定模型**超參數**
    - **控制隨機亂數**
- 計算**準確度（Accuracy）**
    - 使用 **5-fold cross validation**，輸出**平均準確度**
    - 使用 **80%訓練資料**訓練模型、以**20%資料**進行驗證，並輸出**準確度**

In [ ]:
# 模型訓練

from sklearn.model_selection import KFold             # 匯入 K 次交叉驗證工具
from sklearn.tree import DecisionTreeClassifier       # 匯入決策樹模型
from sklearn.metrics import accuracy_score            # 匯入準確度計算工具

kf = KFold(n_splits=5,                                # 設定 K 值
           random_state=1012,
           shuffle=True)
kf.get_n_splits(train_x)                              # 給予資料範圍

train_acc_list = []                                   # 儲存每次訓練模型的準確度
valid_acc_list = []                                   # 儲存每次驗證模型的準確度

for train_index, valid_index in kf.split(train_x):    # 每個迴圈都會產生不同部份的資料
    train_x_split = train_x.iloc[train_index]         # 產生訓練資料
    train_y_split = train_y.iloc[train_index]         # 產生訓練資料標籤
    valid_x_split = train_x.iloc[valid_index]         # 產生驗證資料
    valid_y_split = train_y.iloc[valid_index]         # 產生驗證資料標籤
    
    model = DecisionTreeClassifier(random_state=1012) # 創造決策樹模型
    model.fit(train_x_split, train_y_split)           # 訓練決策樹模型
    
    train_pred_y = model.predict(train_x_split)       # 確認模型是否訓練成功
    train_acc = accuracy_score(train_y_split,         # 計算訓練資料準確度
                               train_pred_y)
    valid_pred_y = model.predict(valid_x_split)       # 驗證模型是否訓練成功
    valid_acc = accuracy_score(valid_y_split,         # 計算驗證資料準確度
                               valid_pred_y)
    
    train_acc_list.append(train_acc)
    valid_acc_list.append(valid_acc)

print((
    'average train accuracy: {}\n' +
    '    min train accuracy: {}\n' +
    '    max train accuracy: {}\n' +
    'average valid accuracy: {}\n' +
    '    min valid accuracy: {}\n' +
    '    max valid accuracy: {}').format(
    np.mean(train_acc_list),                          # 輸出平均訓練準確度
    np.min(train_acc_list),                           # 輸出最低訓練準確度
    np.max(train_acc_list),                           # 輸出最高訓練準確度
    np.mean(valid_acc_list),                          # 輸出平均驗證準確度
    np.min(valid_acc_list),                           # 輸出最低驗證準確度
    np.max(valid_acc_list)                            # 輸出最高驗證準確度
))

## 額外練習

請將訓練後的模型套用至 [Kaggle Titanic](https://www.kaggle.com/c/titanic/overview) `test.csv` 上，並上傳至 Kaggle 進行評分。

In [ ]:
# 請勿更動此區塊程式碼

EXECUTION_END_TIME = time.time() # 計算執行時間
print('total execution time: {}'.format(EXECUTION_END_TIME - EXECUTION_START_TIME))